In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import torch

In [7]:
#TODO
#load in swat and wadi datasets and match matrix indexes with the column names
#create classes
#calculate homophily with sensor classes

In [8]:
wadi_sensors = [
    "Row", "Date", "Time", "1_AIT_001_PV", "1_AIT_002_PV", "1_AIT_003_PV",
    "1_AIT_004_PV", "1_AIT_005_PV", "1_FIT_001_PV", "1_LS_001_AL", "1_LS_002_AL",
    "1_LT_001_PV", "1_MV_001_STATUS", "1_MV_002_STATUS", "1_MV_003_STATUS",
    "1_MV_004_STATUS", "1_P_001_STATUS", "1_P_002_STATUS", "1_P_003_STATUS",
    "1_P_004_STATUS", "1_P_005_STATUS", "1_P_006_STATUS", "2_DPIT_001_PV",
    "2_FIC_101_CO", "2_FIC_101_PV", "2_FIC_101_SP", "2_FIC_201_CO", "2_FIC_201_PV",
    "2_FIC_201_SP", "2_FIC_301_CO", "2_FIC_301_PV", "2_FIC_301_SP", "2_FIC_401_CO",
    "2_FIC_401_PV", "2_FIC_401_SP", "2_FIC_501_CO", "2_FIC_501_PV", "2_FIC_501_SP",
    "2_FIC_601_CO", "2_FIC_601_PV", "2_FIC_601_SP", "2_FIT_001_PV", "2_FIT_002_PV",
    "2_FIT_003_PV", "2_FQ_101_PV", "2_FQ_201_PV", "2_FQ_301_PV", "2_FQ_401_PV",
    "2_FQ_501_PV", "2_FQ_601_PV", "2_LS_001_AL", "2_LS_002_AL", "2_LS_101_AH",
    "2_LS_101_AL", "2_LS_201_AH", "2_LS_201_AL", "2_LS_301_AH", "2_LS_301_AL",
    "2_LS_401_AH", "2_LS_401_AL", "2_LS_501_AH", "2_LS_501_AL", "2_LS_601_AH",
    "2_LS_601_AL", "2_LT_001_PV", "2_LT_002_PV", "2_MCV_007_CO", "2_MCV_101_CO",
    "2_MCV_201_CO", "2_MCV_301_CO", "2_MCV_401_CO", "2_MCV_501_CO", "2_MCV_601_CO",
    "2_MV_001_STATUS", "2_MV_002_STATUS", "2_MV_003_STATUS", "2_MV_004_STATUS",
    "2_MV_005_STATUS", "2_MV_006_STATUS", "2_MV_009_STATUS", "2_MV_101_STATUS",
    "2_MV_201_STATUS", "2_MV_301_STATUS", "2_MV_401_STATUS", "2_MV_501_STATUS",
    "2_MV_601_STATUS", "2_P_001_STATUS", "2_P_002_STATUS", "2_P_003_SPEED",
    "2_P_003_STATUS", "2_P_004_SPEED", "2_P_004_STATUS", "2_PIC_003_CO",
    "2_PIC_003_PV", "2_PIC_003_SP", "2_PIT_001_PV", "2_PIT_002_PV", "2_PIT_003_PV",
    "2_SV_101_STATUS", "2_SV_201_STATUS", "2_SV_301_STATUS", "2_SV_401_STATUS",
    "2_SV_501_STATUS", "2_SV_601_STATUS", "2A_AIT_001_PV", "2A_AIT_002_PV",
    "2A_AIT_003_PV", "2A_AIT_004_PV", "2B_AIT_001_PV", "2B_AIT_002_PV",
    "2B_AIT_003_PV", "2B_AIT_004_PV", "3_AIT_001_PV", "3_AIT_002_PV",
    "3_AIT_003_PV", "3_AIT_004_PV", "3_AIT_005_PV", "3_FIT_001_PV", "3_LS_001_AL",
    "3_LT_001_PV", "3_MV_001_STATUS", "3_MV_002_STATUS", "3_MV_003_STATUS",
    "3_P_001_STATUS", "3_P_002_STATUS", "3_P_003_STATUS", "3_P_004_STATUS",
    "LEAK_DIFF_PRESSURE", "PLANT_START_STOP_LOG", "TOTAL_CONS_REQUIRED_FLOW"
]
swat_sensors = [
    "Timestamp", "FIT101", "LIT101", "MV101", "P101", "P102", "AIT201", "AIT202",
    "AIT203", "FIT201", "MV201", "P201", "P202", "P203", "P204", "P205", "P206",
    "DPIT301", "FIT301", "LIT301", "MV301", "MV302", "MV303", "MV304", "P301",
    "P302", "AIT401", "AIT402", "FIT401", "LIT401", "P401", "P402", "P403", "P404",
    "UV401", "AIT501", "AIT502", "AIT503", "AIT504", "FIT501", "FIT502", "FIT503",
    "FIT504", "P501", "P502", "PIT501", "PIT502", "PIT503", "FIT601", "P601",
    "P602", "P603", "Normal/Attack"
]

# Create a dictionary with index as key and [sensor_name, type, group] as value
swat_sensors_dict = {
    0: ["FIT101", "Sensor", 1],
    1: ["LIT101", "Sensor", 1],
    2: ["MV101", "Actuator", 1],
    3: ["P101", "Actuator", 1],
    4: ["P102", "Actuator", 1],
    5: ["AIT201", "Sensor", 2],
    6: ["AIT202", "Sensor", 2],
    7: ["AIT203", "Sensor", 2],
    8: ["FIT201", "Sensor", 2],
    9: ["MV201", "Actuator", 2],
    10: ["P201", "Actuator", 2],
    11: ["P202", "Actuator", 2],
    12: ["P203", "Actuator", 2],
    13: ["P204", "Actuator", 2],
    14: ["P205", "Actuator", 2],
    15: ["P206", "Actuator", 2],
    16: ["DPIT301", "Sensor", 3],
    17: ["FIT301", "Sensor", 3],
    18: ["LIT301", "Sensor", 3],
    19: ["MV301", "Actuator", 3],
    20: ["MV302", "Actuator", 3],
    21: ["MV303", "Actuator", 3],
    22: ["MV304", "Actuator", 3],
    23: ["P301", "Actuator", 3],
    24: ["P302", "Actuator", 3],
    25: ["AIT401", "Sensor", 4],
    26: ["AIT402", "Sensor", 4],
    27: ["FIT401", "Sensor", 4],
    28: ["LIT401", "Actuator", 4],  # Note: LIT401 is listed as an Actuator in your data, though LIT typically indicates a sensor
    29: ["P401", "Actuator", 4],
    30: ["P402", "Actuator", 4],
    31: ["P403", "Actuator", 4],
    32: ["P404", "Actuator", 4],
    33: ["UV401", "Actuator", 4],
    34: ["AIT501", "Sensor", 5],
    35: ["AIT502", "Sensor", 5],
    36: ["AIT503", "Sensor", 5],
    37: ["AIT504", "Sensor", 5],
    38: ["FIT501", "Sensor", 5],
    39: ["FIT502", "Sensor", 5],
    40: ["FIT503", "Sensor", 5],
    41: ["FIT504", "Sensor", 5],
    42: ["P501", "Actuator", 5],
    43: ["P502", "Actuator", 5],
    44: ["PIT501", "Sensor", 5],
    45: ["PIT502", "Sensor", 5],
    46: ["PIT503", "Sensor", 5],
    47: ["FIT601", "Sensor", 6],
    48: ["P601", "Actuator", 6],
    49: ["P602", "Actuator", 6],
    50: ["P603", "Actuator", 6]
}

wadi_sensors_dict = {
    # Primary Grid (P1)
    0: ["1_AIT_001_PV", "Sensor", "P1"],  # Water quality sensor
    1: ["1_AIT_002_PV", "Sensor", "P1"],  # Water quality sensor
    2: ["1_AIT_003_PV", "Sensor", "P1"],  # Water quality sensor
    3: ["1_AIT_004_PV", "Sensor", "P1"],  # Water quality sensor
    4: ["1_AIT_005_PV", "Sensor", "P1"],  # Water quality sensor
    5: ["1_FIT_001_PV", "Sensor", "P1"],  # Flow meter
    6: ["1_LS_001_AL", "Sensor", "P1"],   # Level switch alarm
    7: ["1_LS_002_AL", "Sensor", "P1"],   # Level switch alarm
    8: ["1_LT_001_PV", "Sensor", "P1"],   # Level transmitter
    9: ["1_MV_001_STATUS", "Actuator", "P1"],  # Motorized valve
    10: ["1_MV_002_STATUS", "Actuator", "P1"],  # Motorized valve
    11: ["1_MV_003_STATUS", "Actuator", "P1"],  # Motorized valve
    12: ["1_MV_004_STATUS", "Actuator", "P1"],  # Motorized valve
    13: ["1_P_001_STATUS", "Actuator", "P1"],   # Pump
    14: ["1_P_002_STATUS", "Actuator", "P1"],   # Pump
    15: ["1_P_003_STATUS", "Actuator", "P1"],   # Pump
    16: ["1_P_004_STATUS", "Actuator", "P1"],   # Pump
    17: ["1_P_005_STATUS", "Actuator", "P1"],   # Pump
    18: ["1_P_006_STATUS", "Actuator", "P1"],   # Pump

    # Secondary Grid Part A (P2A - Elevated Reservoir)
    19: ["2_DPIT_001_PV", "Sensor", "P2A"],  # Differential pressure indicator
    20: ["2_FIT_001_PV", "Sensor", "P2A"],   # Flow meter
    21: ["2_FIT_002_PV", "Sensor", "P2A"],   # Flow meter
    22: ["2_FIT_003_PV", "Sensor", "P2A"],   # Flow meter
    23: ["2_LT_001_PV", "Sensor", "P2A"],    # Level transmitter - elevated reservoir
    24: ["2_LT_002_PV", "Sensor", "P2A"],    # Level transmitter - elevated reservoir
    25: ["2_LS_001_AL", "Sensor", "P2A"],    # Level switch alarm (low) for elevated reservoir tanks
    26: ["2_LS_002_AL", "Sensor", "P2A"],    # Level switch alarm (low) for elevated reservoir tanks
    27: ["2_MV_001_STATUS", "Actuator", "P2A"],  # Motorized valve
    28: ["2_MV_002_STATUS", "Actuator", "P2A"],  # Motorized valve
    29: ["2_MV_003_STATUS", "Actuator", "P2A"],  # Motorized valve
    30: ["2_MV_004_STATUS", "Actuator", "P2A"],  # Motorized valve
    31: ["2_MV_005_STATUS", "Actuator", "P2A"],  # Motorized valve
    32: ["2_MV_006_STATUS", "Actuator", "P2A"],  # Motorized valve
    33: ["2_MV_009_STATUS", "Actuator", "P2A"],  # Motorized valve
    34: ["2_P_001_STATUS", "Actuator", "P2A"],   # Pump
    35: ["2_P_002_STATUS", "Actuator", "P2A"],   # Pump
    36: ["2_P_003_SPEED", "Actuator", "P2A"],    # Speed control for pump 3
    37: ["2_P_003_STATUS", "Actuator", "P2A"],   # Status of pump 3
    38: ["2_P_004_SPEED", "Actuator", "P2A"],    # Speed control for pump 4
    39: ["2_P_004_STATUS", "Actuator", "P2A"],   # Status of pump 4
    40: ["2_PIC_003_CO", "Actuator", "P2A"],     # Control output for pressure controller 3
    41: ["2_PIC_003_PV", "Sensor", "P2A"],       # Process value for pressure controller 3
    42: ["2_PIC_003_SP", "Sensor", "P2A"],       # Setpoint for pressure controller 3
    43: ["2_PIT_001_PV", "Sensor", "P2A"],       # Pressure indicator
    44: ["2_PIT_002_PV", "Sensor", "P2A"],       # Pressure indicator
    45: ["2_PIT_003_PV", "Sensor", "P2A"],       # Pressure indicator
    46: ["2A_AIT_001_PV", "Sensor", "P2A"],      # Water quality sensor
    47: ["2A_AIT_002_PV", "Sensor", "P2A"],      # Water quality sensor
    48: ["2A_AIT_003_PV", "Sensor", "P2A"],      # Water quality sensor
    49: ["2A_AIT_004_PV", "Sensor", "P2A"],      # Water quality sensor

    # Secondary Grid Part B (P2B - Consumer Tanks)
    50: ["2_FIC_101_PV", "Sensor", "P2B"],       # Flow indicator/controller process value
    51: ["2_FIC_101_CO", "Actuator", "P2B"],     # Control output for flow controller 101
    52: ["2_FIC_101_SP", "Sensor", "P2B"],       # Setpoint for flow controller 101
    53: ["2_FIC_201_PV", "Sensor", "P2B"],       # Flow indicator/controller process value
    54: ["2_FIC_201_CO", "Actuator", "P2B"],     # Control output for flow controller 201
    55: ["2_FIC_201_SP", "Sensor", "P2B"],       # Setpoint for flow controller 201
    56: ["2_FIC_301_PV", "Sensor", "P2B"],       # Flow indicator/controller process value
    57: ["2_FIC_301_CO", "Actuator", "P2B"],     # Control output for flow controller 301
    58: ["2_FIC_301_SP", "Sensor", "P2B"],       # Setpoint for flow controller 301
    59: ["2_FIC_401_PV", "Sensor", "P2B"],       # Flow indicator/controller process value
    60: ["2_FIC_401_CO", "Actuator", "P2B"],     # Control output for flow controller 401
    61: ["2_FIC_401_SP", "Sensor", "P2B"],       # Setpoint for flow controller 401
    62: ["2_FIC_501_PV", "Sensor", "P2B"],       # Flow indicator/controller process value
    63: ["2_FIC_501_CO", "Actuator", "P2B"],     # Control output for flow controller 501
    64: ["2_FIC_501_SP", "Sensor", "P2B"],       # Setpoint for flow controller 501
    65: ["2_FIC_601_PV", "Sensor", "P2B"],       # Flow indicator/controller process value
    66: ["2_FIC_601_CO", "Actuator", "P2B"],     # Control output for flow controller 601
    67: ["2_FIC_601_SP", "Sensor", "P2B"],       # Setpoint for flow controller 601
    68: ["2_FQ_101_PV", "Sensor", "P2B"],        # Flow totalizer
    69: ["2_FQ_201_PV", "Sensor", "P2B"],        # Flow totalizer
    70: ["2_FQ_301_PV", "Sensor", "P2B"],        # Flow totalizer
    71: ["2_FQ_401_PV", "Sensor", "P2B"],        # Flow totalizer
    72: ["2_FQ_501_PV", "Sensor", "P2B"],        # Flow totalizer
    73: ["2_FQ_601_PV", "Sensor", "P2B"],        # Flow totalizer
    74: ["2_LS_101_AH", "Sensor", "P2B"],        # Level switch high alarm
    75: ["2_LS_101_AL", "Sensor", "P2B"],        # Level switch low alarm
    76: ["2_LS_201_AH", "Sensor", "P2B"],        # Level switch high alarm
    77: ["2_LS_201_AL", "Sensor", "P2B"],        # Level switch low alarm
    78: ["2_LS_301_AH", "Sensor", "P2B"],        # Level switch high alarm
    79: ["2_LS_301_AL", "Sensor", "P2B"],        # Level switch low alarm
    80: ["2_LS_401_AH", "Sensor", "P2B"],        # Level switch high alarm
    81: ["2_LS_401_AL", "Sensor", "P2B"],        # Level switch low alarm
    82: ["2_LS_501_AH", "Sensor", "P2B"],        # Level switch high alarm
    83: ["2_LS_501_AL", "Sensor", "P2B"],        # Level switch low alarm
    84: ["2_LS_601_AH", "Sensor", "P2B"],        # Level switch high alarm
    85: ["2_LS_601_AL", "Sensor", "P2B"],        # Level switch low alarm
    86: ["2_MCV_007_CO", "Actuator", "P2B"],     # Control output for motorized consumer valve 7
    87: ["2_MCV_101_CO", "Actuator", "P2B"],     # Control output for motorized consumer valve 101
    88: ["2_MCV_201_CO", "Actuator", "P2B"],     # Control output for motorized consumer valve 201
    89: ["2_MCV_301_CO", "Actuator", "P2B"],     # Control output for motorized consumer valve 301
    90: ["2_MCV_401_CO", "Actuator", "P2B"],     # Control output for motorized consumer valve 401
    91: ["2_MCV_501_CO", "Actuator", "P2B"],     # Control output for motorized consumer valve 501
    92: ["2_MCV_601_CO", "Actuator", "P2B"],     # Control output for motorized consumer valve 601
    93: ["2_MV_101_STATUS", "Actuator", "P2B"],  # Status of motorized valve 101 in consumer section
    94: ["2_MV_201_STATUS", "Actuator", "P2B"],  # Status of motorized valve 201 in consumer section
    95: ["2_MV_301_STATUS", "Actuator", "P2B"],  # Status of motorized valve 301 in consumer section
    96: ["2_MV_401_STATUS", "Actuator", "P2B"],  # Status of motorized valve 401 in consumer section
    97: ["2_MV_501_STATUS", "Actuator", "P2B"],  # Status of motorized valve 501 in consumer section
    98: ["2_MV_601_STATUS", "Actuator", "P2B"],  # Status of motorized valve 601 in consumer section
    99: ["2_SV_101_STATUS", "Actuator", "P2B"],  # Solenoid valve
    100: ["2_SV_201_STATUS", "Actuator", "P2B"], # Solenoid valve
    101: ["2_SV_301_STATUS", "Actuator", "P2B"], # Solenoid valve
    102: ["2_SV_401_STATUS", "Actuator", "P2B"], # Solenoid valve
    103: ["2_SV_501_STATUS", "Actuator", "P2B"], # Solenoid valve
    104: ["2_SV_601_STATUS", "Actuator", "P2B"], # Solenoid valve
    105: ["2B_AIT_001_PV", "Sensor", "P2B"],     # Water quality sensor
    106: ["2B_AIT_002_PV", "Sensor", "P2B"],     # Water quality sensor
    107: ["2B_AIT_003_PV", "Sensor", "P2B"],     # Water quality sensor
    108: ["2B_AIT_004_PV", "Sensor", "P2B"],     # Water quality sensor

    # Return Water Grid (P3)
    109: ["3_AIT_001_PV", "Sensor", "P3"],       # Water quality sensor
    110: ["3_AIT_002_PV", "Sensor", "P3"],       # Water quality sensor
    111: ["3_AIT_003_PV", "Sensor", "P3"],       # Water quality sensor
    112: ["3_AIT_004_PV", "Sensor", "P3"],       # Water quality sensor
    113: ["3_AIT_005_PV", "Sensor", "P3"],       # Water quality sensor
    114: ["3_FIT_001_PV", "Sensor", "P3"],       # Flow meter
    115: ["3_LS_001_AL", "Sensor", "P3"],        # Level switch alarm
    116: ["3_LT_001_PV", "Sensor", "P3"],        # Level transmitter
    117: ["3_MV_001_STATUS", "Actuator", "P3"],  # Motorized valve
    118: ["3_MV_002_STATUS", "Actuator", "P3"],  # Motorized valve
    119: ["3_MV_003_STATUS", "Actuator", "P3"],  # Motorized valve
    120: ["3_P_001_STATUS", "Actuator", "P3"],   # Pump
    121: ["3_P_002_STATUS", "Actuator", "P3"],   # Pump
    122: ["3_P_003_STATUS", "Actuator", "P3"],   # Pump
    123: ["3_P_004_STATUS", "Actuator", "P3"],   # Pump

    # System-wide parameters
    124: ["LEAK_DIFF_PRESSURE", "Sensor", "System"],      # Differential pressure used to detect leaks
    125: ["PLANT_START_STOP_LOG", "Sensor", "System"],    # System-wide log for plant startup/shutdown
    126: ["TOTAL_CONS_REQUIRED_FLOW", "Sensor", "System"] # Total required consumer flow across all tanks
}

# Example usage
# print(wadi_dict[5])  # Output: ['1_FIT_001_PV', 'Sensor', 'P1']
# print(wadi_dict[51]) # Output: ['2_FIC_101_CO', 'Actuator', 'P2B']


In [9]:
node_num_swat=51
node_num_wadi=128
# swat_gdn_graph_path='/berentzenaej/baselines/GDN/results/swat_learned_graph*.txt'
# wadi_gdn_graph_path='/berentzenaej/baselines/GDN/results/wadi_learned_graph*.txt'

swat_cstgl_graph_path='/home/berentzenaej/baselines/CST-GL/save/topk_adjacency_swat.txt'
wadi_cstgl_graph_path='/home/berentzenaej/baselines/CST-GL/save/topk_adjacency_wadi.txt'



In [10]:
#structural
def build_tensor(dataset):
    # Expand the tilde to the home directory
    pattern = os.path.expanduser(f"/home/berentzenaej/baselines/CST-GL/save/topk_adjacency_{dataset}*.txt")

    # Find all matching files
    file_paths = sorted(glob.glob(pattern))
    print(f"Found {len(file_paths)} files matching the pattern")

    # Function to load a matrix from a text file
    def load_matrix(file_path):
        return np.loadtxt(file_path)

    # Load all matrices into a tensor
    matrices = []
    for file_path in file_paths:
        matrix = load_matrix(file_path)
        matrices.append(matrix)

    # Stack all matrices into a single tensor
    if matrices:
        learned_graph_tensor = np.stack(matrices)
        print(f"Created tensor with shape {learned_graph_tensor.shape}")
    else:
        print("No matrices found")
    return learned_graph_tensor

swat_tensor=build_tensor('swat')
wadi_tensor=build_tensor('wadi')


Found 5 files matching the pattern
Created tensor with shape (5, 51, 16)
Found 8 files matching the pattern
Created tensor with shape (8, 127, 31)


In [11]:
def calculate_jaccard(graph_tensor):
    num_graphs = graph_tensor.shape[0]
    if num_graphs <= 1:
        return 0.0

    edge_sets = []
    for i in range(num_graphs):
        edges = set()
        graph = graph_tensor[i]
        for src in range(len(graph)):
            for tgt in graph[src]:
                if tgt >= 0:
                    edges.add((src, int(tgt)))
        edge_sets.append(edges)

    total_jaccard = 0.0
    pair_count = 0
    for i in range(num_graphs):
        for j in range(i+1, num_graphs):
            intersection = len(edge_sets[i] & edge_sets[j])
            union = len(edge_sets[i] | edge_sets[j])
            jaccard = 1.0 if union == 0 else intersection / union
            total_jaccard += jaccard
            pair_count += 1

    return total_jaccard / pair_count if pair_count > 0 else 0.0

In [12]:
calculate_jaccard(swat_tensor)

0.26754183468569537

In [13]:
calculate_jaccard(wadi_tensor)

0.26963898265706226

In [14]:
def calculate_edge_occurrence(tensor):
    """
    Calculate directed edge occurrence counts across matrices.

    Args:
        tensor: numpy array of shape (num_matrices, num_nodes, max_neighbors)

    Returns:
        dict: {(node_i, node_j): count} edge occurrence dictionary
    """
    edge_counts = {}

    # Iterate through each matrix
    for m in range(tensor.shape[0]):
        # Iterate through each node
        for i in range(tensor.shape[1]):
            # Iterate through each neighbor index
            for j_idx in range(tensor.shape[2]):
                j = tensor[m, i, j_idx]

                # Create edge tuple and increment count
                edge = (i, int(j))
                edge_counts[edge] = edge_counts.get(edge, 0) + 1

    return edge_counts
wadi_occurence_dict=calculate_edge_occurrence(wadi_tensor)
swat_occurence_dict=calculate_edge_occurrence(swat_tensor)


In [15]:
swat_occurence_list_sorted=sorted(swat_occurence_dict.items(), key=lambda x: x[1], reverse=True)
wadi_occurence_list_sorted=sorted(wadi_occurence_dict.items(), key=lambda x: x[1], reverse=True)

### This is for identifying the most consistent edges

In [16]:
def top_different_pairs(occurence_list, node_info):
    # Filter to keep only pairs with different nodes
    different_pairs = {pair: count for pair, count in occurence_list if pair[0] != pair[1]}

    # Sort by count (descending) and get top 5
    top_pairs = sorted(different_pairs.items(), key=lambda x: x[1], reverse=True)[:5]

    # Return result with node info
    return [(pair, count, (node_info[pair[0]], node_info[pair[1]]))
            for pair, count in top_pairs]

In [17]:
wadi_top_pairs = top_different_pairs(wadi_occurence_list_sorted,wadi_sensors_dict)
swat_top_pairs = top_different_pairs(swat_occurence_list_sorted,swat_sensors_dict)

In [18]:
swat_top_pairs

[((0, 20), 5, (['FIT101', 'Sensor', 1], ['MV302', 'Actuator', 3])),
 ((2, 20), 5, (['MV101', 'Actuator', 1], ['MV302', 'Actuator', 3])),
 ((3, 11), 5, (['P101', 'Actuator', 1], ['P202', 'Actuator', 2])),
 ((4, 20), 5, (['P102', 'Actuator', 1], ['MV302', 'Actuator', 3])),
 ((5, 20), 5, (['AIT201', 'Sensor', 2], ['MV302', 'Actuator', 3]))]

In [22]:
def calculate_homophily(adj_tensor, sensor_dictionary, type=1):
    """
    Calculate graph homophily using the formula:
    h = (sum_{i in V} sum_{j in N(i)} 1(c_i = c_j)) / (2|E|)

    Parameters:
    -----------
    adj_tensor : list of lists of lists
        3D tensor where:
        - First dimension: different graphs
        - Second dimension: nodes in each graph
        - Third dimension: connections for each node (padded with -1)
    node_attributes : dict
        Dictionary mapping node indices to their attributes.
    type : int
        type of homophily calculation (based on sensor/actuator or group)
        (1 for Type, 2 for Group).

    Returns:
    --------
    list
        List of homophily values for each graph in the tensor.
    """
    # List to store homophily values for each graph
    homophily_values = []

    # Process each graph in the tensor
    for graph_idx, graph in enumerate(adj_tensor):
        # Count total edges and same-attribute edges for this graph
        total_edges = 0
        same_attribute_edges = 0

        # Iterate through each node and its neighbors
        for node_i, neighbors in enumerate(graph):
            # Get attribute for current node
            attr_i = sensor_dictionary[node_i][type]

            # Iterate through all neighbors of node_i
            for node_j in neighbors:
                # Skip padding values (-1)
                if node_j == -1:
                    continue

                # Get attribute for neighbor node
                attr_j = sensor_dictionary[node_j][type]

                # Increment total edges
                total_edges += 1

                # Check if attributes match
                if attr_i == attr_j:
                    same_attribute_edges += 1

        # Calculate homophily for this graph
        graph_homophily = same_attribute_edges / total_edges if total_edges > 0 else 0
        homophily_values.append(graph_homophily)

    return homophily_values
print(f'swat sensor/actuator {calculate_homophily(swat_tensor, swat_sensors_dict,1)}')


KeyboardInterrupt: 

In [20]:
print(f'swat sensor/actuator {calculate_homophily(swat_tensor, swat_sensors_dict,1)}')
print(f'swat fase {calculate_homophily(swat_tensor, swat_sensors_dict,2)}')
print(f'wadi sensor/actuator {calculate_homophily(wadi_tensor, wadi_sensors_dict,1)}')
print(f'wadi fase {calculate_homophily(wadi_tensor,wadi_sensors_dict,2)}')


swat sensor/actuator [0.5465686274509803, 0.5110294117647058, 0.5257352941176471, 0.5073529411764706, 0.5318627450980392]
swat fase [0.23161764705882354, 0.23651960784313725, 0.24754901960784315, 0.20588235294117646, 0.22426470588235295]
wadi sensor/actuator [0.5267970535941072, 0.523241046482093, 0.5189230378460757, 0.5267970535941072, 0.5267970535941072, 0.5090170180340361, 0.5275590551181102, 0.5166370332740665]
wadi fase [0.33096266192532386, 0.3543307086614173, 0.30226060452120906, 0.33096266192532386, 0.33096266192532386, 0.3213106426212852, 0.28422656845313693, 0.3243586487172974]


In [21]:
print(f'swat sensor/actuator {np.mean(calculate_homophily(swat_tensor, swat_sensors_dict,1))}')
print(f'swat fase {np.mean(calculate_homophily(swat_tensor, swat_sensors_dict,2))}')
print(f'wadi sensor/actuator {np.mean(calculate_homophily(wadi_tensor, wadi_sensors_dict,1))}')
print(f'wadi fase {np.mean(calculate_homophily(wadi_tensor,wadi_sensors_dict,2))}')

swat sensor/actuator 0.5245098039215687
swat fase 0.22916666666666669
wadi sensor/actuator 0.5219710439420879
wadi fase 0.3224218948437897
